## 0. Предварительные действия

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# импортируем библиотеки для визуализации
import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

In [ ]:
DATA_DIR = 'D:/Python/IDE/project_3/data'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'/hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
# Подгрузим наши данные из соревнования

#DATA_DIR = '/kaggle/input/sf-booking/'
#df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
#df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
#sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

In [ ]:
df_train.info()

In [ ]:
df_train.head(2)

In [ ]:
df_test.info()

In [ ]:
df_test.head(2)

In [ ]:
sample_submission.head(2)

In [ ]:
sample_submission.info()

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

hotels = pd.concat([df_test, df_train], sort=False).reset_index(drop=True) # объединяем

In [ ]:
hotels.info()

In [ ]:
import dtale
d=dtale.show(hotels)
d.open_browser()

## 1. Базовая проверка качества датасета

In [ ]:
# Ищем выводим колонки с пропуском
cols_null_percent = hotels.isnull().mean()*100
cols_with_null = cols_null_percent[cols_null_percent > 0].sort_values(ascending=False)
print(cols_with_null)

In [ ]:
# Временно заполняем нулями, но мы ещё вернёмся к этому
hotels['lat'] = hotels['lat'].fillna(0)
hotels['lng'] = hotels['lng'].fillna(0)

In [ ]:
# Сводка уникальных значений, разбираемся, что у нас будет категориальным признаком
hotels.nunique(dropna=False)

In [ ]:
# Ищем дубликаты
dupl_col=list(hotels.columns)
mask=hotels.duplicated(subset=dupl_col)
data_doubles=hotels[mask]
data_doubles.shape
# В таблице имеется 336 дубликатов. Однако, по условиям задачи удалять строки нельзя, так что пока оставим как есть.

## 2. Работаем со столбцами

In [ ]:
hotels.info()

### hotel_address

In [ ]:
# Работаем с адресом отеля (страна)
import pycountry
countries = {c.name for c in pycountry.countries} | {c.official_name for c in pycountry.countries if hasattr(c, "official_name")} | {c.alpha_3 for c in pycountry.countries} | {c.alpha_2 for c in pycountry.countries}

def extract_country(address):
    for country in countries:
        if address.endswith(country):
            return country
    return None

# Применяем к колонке
hotels["country"] = hotels["hotel_address"].apply(extract_country)

In [ ]:
# Смотрим, что получилось
hotels.country.value_counts()
# Получилось немного стран, попробуем выделить города

In [ ]:
# вытаскиваем город из адреса
def extract_city(addr):
    parts = addr.split()
    if parts[-2:] == ['United', 'Kingdom']:
        return parts[-5]  # для UK формаат адреса город код код United Kingdom
    else:
        return parts[-2]  # для всех остальных город перед страной

# Создаем признак city
hotels["city"] = hotels["hotel_address"].apply(extract_city)


In [ ]:
hotels['city'].value_counts()

Пользы нет - в датасете все гостиницы из одной страны находятся в одном городе. Пока не удаляем - используем эти данные для лучшего заполнения координат

### Дата ревью

In [ ]:
#Преобразуем в datetime
hotels['review_date'] = pd.to_datetime(hotels['review_date'], dayfirst=True, errors='coerce')

# Создаём признаки год, месяц, день недели и является ли день выходным. Дата будет очевидно не информативным признаком, вместо неё берём день недели
# и статус выходного (теория - "платные" накрутки чаще происходят в будний день)
hotels['year'] = hotels['review_date'].dt.year
hotels['month'] = hotels['review_date'].dt.month
hotels['dayofweek'] = hotels['review_date'].dt.weekday  # 0 = понедельник, 6 = воскресенье
hotels['is_weekend'] = (hotels['dayofweek'] >= 5).astype('int8')


In [ ]:
# Проверям все ли получилось
hotels[['year', 'month', 'dayofweek', 'is_weekend']].head()

### hotel_name 

In [ ]:
hotels['hotel_name'].describe()

In [ ]:
# Попробуем закодировать с помощью frequency и targetencoding
from sklearn.model_selection import KFold

# freq
freq = hotels['hotel_name'].value_counts()
hotels['hotel_name_freq'] = hotels['hotel_name'].map(freq).astype('int32')

# target
kf = KFold(n_splits=5, shuffle=True, random_state=42)
te = np.zeros(len(hotels))
for tr, va in kf.split(hotels):
    m = hotels.iloc[tr].groupby('hotel_name')['reviewer_score'].mean()
    te[va] = hotels.iloc[va]['hotel_name'].map(m)
hotels['hotel_name_te'] = te

In [ ]:
# Альтернатива - довичное кодирование с помощью category_encoders
#import category_encoders as ce
#bin_encoder = ce.BinaryEncoder(cols=['hotel_name'])
#data_bin=bin_encoder.fit_transform(hotels['hotel_name'])
#hotels = pd.concat([hotels, data_bin], axis=1)

### reviewer_nationality

In [ ]:
# Убираем пробелы в национальности
hotels["reviewer_nationality"] = hotels["reviewer_nationality"].str.strip()

In [ ]:
a=hotels['reviewer_nationality'].value_counts()
print(len(a))

In [ ]:
# Frequency encoding (просто количество записей по стране)
freq = hotels['reviewer_nationality'].value_counts()
hotels['reviewer_nationality_freq'] = hotels['reviewer_nationality'].map(freq)

# Target encoding
kf = KFold(n_splits=5, shuffle=True, random_state=42)
te = np.zeros(len(hotels))

for train_idx, val_idx in kf.split(hotels):
    means = hotels.iloc[train_idx].groupby('reviewer_nationality')['reviewer_score'].mean()
    te[val_idx] = hotels.iloc[val_idx]['reviewer_nationality'].map(means)

hotels['reviewer_nationality_te'] = te

### negative_review/positive_review

In [ ]:
# Работаем с текстом отрицательного отзыва negative_review через ntlk с полным набором скачанных пакетов
import re
import nltk
nltk.download('vader_lexicon')

from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

def clean_text(s: str) -> str:
    if not isinstance(s, str): 
        return ""
    s = s.strip()
    s = re.sub(r"(https?://\S+)|(\S+@\S+)", " ", s)                     # удалим URL и email
    s = re.sub(r"[^A-Za-z0-9\s\.\,\!\?\-']", " ", s)                    # оставим только валидные символы
    s = re.sub(r"\s+", " ", s).strip()                                  # уберём лишние пробелы
    return s

# Очистка текста
hotels["neg_clean"] = hotels["negative_review"].map(clean_text)

# Признаки
hotels["neg_len_chars"] = hotels["neg_clean"].str.len()
hotels["neg_len_words"] = hotels["neg_clean"].str.split().str.len()
hotels["exclam_cnt"]    = hotels["neg_clean"].str.count(r"!")
hotels["quest_cnt"]     = hotels["neg_clean"].str.count(r"\?")
hotels["caps_ratio"]    = hotels["negative_review"].map(
    lambda s: (sum(c.isupper() for c in s) / max(1, len(s))) if isinstance(s, str) else 0.0
)
hotels["neg_vader"] = hotels["neg_clean"].map(lambda s: sia.polarity_scores(s)["compound"])
hotels["is_no_negative"] = hotels["neg_clean"].str.fullmatch(r"(?i)no negative").fillna(False).astype("int8") # если отзыв состоит из фразы "No Negative" то он положительный


In [ ]:
sia = SentimentIntensityAnalyzer()

def clean_text(s: str) -> str:
    if not isinstance(s, str): 
        return ""
    s = s.strip()
    s = re.sub(r"(https?://\S+)|(\S+@\S+)", " ", s)
    s = re.sub(r"[^A-Za-z0-9\s\.\,\!\?\-']", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# Очистка текста
hotels["pos_clean"] = hotels["positive_review"].map(clean_text)

# Признаки
hotels["pos_len_chars"] = hotels["pos_clean"].str.len()
hotels["pos_len_words"] = hotels["pos_clean"].str.split().str.len()
hotels["pos_exclam_cnt"] = hotels["pos_clean"].str.count(r"!")
hotels["pos_quest_cnt"]  = hotels["pos_clean"].str.count(r"\?")
hotels["pos_caps_ratio"] = hotels["positive_review"].map(
    lambda s: (sum(c.isupper() for c in s) / max(1, len(s))) if isinstance(s, str) else 0.0
)
hotels["pos_vader"] = hotels["pos_clean"].map(lambda s: sia.polarity_scores(s)["compound"])
hotels["is_no_positive"] = hotels["pos_clean"].str.fullmatch(r"(?i)no positive").fillna(False).astype("int8") # если отзыв состоит из фразы "No Positive" то он отрицательный

### TAGS

In [ ]:
hotels['tags'].value_counts()

Немного поработаем с dtale что бы разобраться, какие в принципе есть теги и отобрали те, которые кажутся наиболее оптимальными

In [ ]:
# Преобразуем колонку 'tags' из строки в список
hotels['tags'] = hotels['tags'].apply(eval)

In [ ]:
# Функция очистки тегов - убираем пробелы по краям и заменяем пробелы внутри тега на _
def clean_tags(tag_list):
    return [tag.strip().replace(" ", "_") for tag in tag_list]

# Применяем очистку
hotels['tags'] = hotels['tags'].apply(clean_tags)

# Собираем все теги в один длинный список
all_tags = [tag for tag_list in hotels['tags'] for tag in tag_list]

In [ ]:
from collections import Counter

tag_counts = Counter(all_tags)
print(tag_counts.most_common(100))

In [ ]:
import re
import ast

# На всякий случай: привести колонку к спискам
def to_list_safe(x):
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        try:
            v = ast.literal_eval(x)
            return v if isinstance(v, list) else [x]
        except:
            return [x]
    return []  # None и прочее

hotels["tags"] = hotels["tags"].apply(to_list_safe)

# Чистим пробелы
hotels["tags"] = hotels["tags"].apply(lambda tags: [t.strip() for t in tags if isinstance(t, str)])

# Поиск подстроки (без учёта регистра)
def has_tag(tags, keyword):
    kw = keyword.lower()
    return int(any(kw in (t or "").lower() for t in tags))

# Бинарные признаки
trip_types = ["Leisure trip", "Business trip"]
companions = ["Couple", "Solo traveler", "Family with young children",
              "Family with older children", "Group"]

for t in trip_types:
    hotels[f"trip_{t.replace(' ', '_').lower()}"] = hotels["tags"].apply(lambda x: has_tag(x, t))

for c in companions:
    hotels[f"companion_{c.replace(' ', '_').lower()}"] = hotels["tags"].apply(lambda x: has_tag(x, c))

# Разбор'Stayed X night(s)' через regex
stay_re = re.compile(r"stayed\s+(\d+)\s+night", re.IGNORECASE)

def stay_length(tags):
    for t in tags:
        if not isinstance(t, str):
            continue
        m = stay_re.search(t)
        if m:
            try:
                num = int(m.group(1))
            except:
                continue
            if num == 1:
                return "short"
            elif num <= 3:
                return "medium"
            elif num <= 7:
                return "week"
            else:
                return "long"
    return None

hotels["stay_length"] = hotels["tags"].apply(stay_length)

# Тип номера
room_keywords = ["Suite", "Apartment", "Studio", "Room"]

def room_type(tags):
    low = [(t or "").lower() for t in tags if isinstance(t, str)]
    for kw in room_keywords:
        if any(kw.lower() in t for t in low):
            return kw
    return None

hotels["room_type"] = hotels["tags"].apply(room_type)


In [ ]:
# Кодируем длину пребывания в отеле и тип номера
import category_encoders as ce
encoder = ce.OneHotEncoder(cols=["stay_length", "room_type"], use_cat_names=True)
hotels = encoder.fit_transform(hotels)  

### days_since_review

In [ ]:
# Работаем с days_since_review
hotels["days_since_review"] = hotels["days_since_review"].str.extract(r'(\d+)').astype(int)

## Попробуем что-то сделать с координатами

In [ ]:
# Попробуем простой способ - если координаты не известны - ставим координаты центра города
# если будет плохо - попробуем геокодером, но он очень медленно будет работать в бесплатной версии
# справочник
city_coords = {
    "London":     (51.5074, -0.1278),
    "Barcelona":  (41.3851, 2.1734),
    "Paris":      (48.8566, 2.3522),
    "Amsterdam":  (52.3676, 4.9041),
    "Vienna":     (48.2100, 16.3738),
    "Milan":      (45.4642, 9.1900)
}

def fill_coords(row):
    if row["lat"] == 0 and row["lng"] == 0:   # если координаты "пустые"
        return city_coords.get(row["city"], (row["lat"], row["lng"]))
    return (row["lat"], row["lng"])

# применяем построчно
hotels[["lat", "lng"]] = hotels.apply(fill_coords, axis=1, result_type="expand")

## Кодируем остатки

In [ ]:
hotels.info()

In [ ]:
# кодируем страну
encoder = ce.OneHotEncoder(cols=["country"], use_cat_names=True)
hotels = encoder.fit_transform(hotels)  

In [ ]:
# кодируем город
encoder = ce.OneHotEncoder(cols=["city"], use_cat_names=True)
hotels = encoder.fit_transform(hotels)

### Дропаем лишнее

In [ ]:
#Удаляем все колонки с типом object
hotels = hotels.drop(columns=[col for col in hotels.columns if hotels[col].dtype == 'object'])

In [ ]:
# удаляем дату обзора
hotels = hotels.drop(columns=['review_date'])

In [ ]:
# Экономим место

# выбираем только int64-колонки
int64_cols = hotels.select_dtypes(include="int64").columns

# фильтруем из них бинарные (только 0/1)
binary_cols = [col for col in int64_cols if hotels[col].dropna().nunique() == 2]

# переводим в int8
hotels = hotels.astype({col: "int8" for col in binary_cols})

In [ ]:
hotels.info()

# Пустая ячейка что бы случайно не зайти дальше, чем надо

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(hotels.drop(['sample'], axis=1).corr(), annot=True)

In [ ]:
# Выводим список мультиколлинеарных признаков
# Берем порог 0,9
def high_corr_pairs(df, corr_thr=0.9):
    corr = df.corr(numeric_only=True)  # только числовые колонки
    pairs = []
    for i in range(len(corr.columns)):
        for j in range(i+1, len(corr.columns)):
            if abs(corr.iloc[i, j]) >= corr_thr:
                pairs.append((corr.columns[i], corr.columns[j], corr.iloc[i, j]))
    return pairs

# пример вызова
pairs = high_corr_pairs(hotels, corr_thr=0.9)
for a, b, c in pairs:
    print(f"{a:20} ~ {b:20} corr={c:.3f}")

Как мы и догадывались - города и страны коррелируют 1 к 1. Удаляем страны.
Есть сильная корреляция между числом слов и символов. Убираем число символов - считаем, что слова показательнее.
убираем число дней после обзора т.к. сильная корреляция с годом
Есть сильная зависимость между числом оценок без отзыва и частотой появления имени отеля. Попробуем убрать число дополнительных отзывов, т.к. имя отеля у нас часть кодирования 

In [ ]:
hotels=hotels.drop(columns=['additional_number_of_scoring', 'review_total_negative_word_counts','review_total_negative_word_counts','review_total_positive_word_counts',
                           'review_total_positive_word_counts','days_since_review', 'country_Italy','country_Netherlands','country_Spain','country_United Kingdom',
                           'country_France','country_Austria','neg_len_chars','pos_len_chars'])

In [ ]:
hotels.info()

In [ ]:
# Теперь выделим тестовую часть
train_data = hotels.query('sample == 1').drop(['sample'], axis=1)
test_data = hotels.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAPE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['reviewer_score'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
list(sample_submission)

In [ ]:
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)